In [1]:
import os
from pyscf import gto, scf, mcscf
from openfermionpyscf import generate_molecular_hamiltonian
from openfermion.transforms import jordan_wigner, bravyi_kitaev_tree
from openfermion.transforms import get_fermion_operator
from IPython.display import display, Markdown
from openfermion.ops import FermionOperator, QubitOperator


# Function to create a directory and save results
def save_results(bond_distance, output_dir, fermion_hamiltonian, jw_transform, bk_transform):
    # Create a folder for the bond distance
    folder_name = os.path.join(output_dir, f"AlH_{int(bond_distance * 100)}")
    os.makedirs(folder_name, exist_ok=True)

    # Save fermionic Hamiltonian
    with open(os.path.join(folder_name, "fermionic_hamiltonian.txt"), "w") as f:
        f.write(str(fermion_hamiltonian))

    # Save Jordan-Wigner transformation
    with open(os.path.join(folder_name, "jordan_wigner.txt"), "w") as f:
        f.write(str(jw_transform))

    # Save Bravyi-Kitaev transformation
    with open(os.path.join(folder_name, "bravyi_kitaev.txt"), "w") as f:
        f.write(str(bk_transform))

    print(f"Results saved in folder: {folder_name}")


# Function to manually compute the Hermitian conjugate of a QubitOperator
def hermitian_conjugate(qubit_operator):
    conjugate_operator = QubitOperator()
    for term, coefficient in qubit_operator.terms.items():
        # Conjugate the coefficient (complex conjugate) and add the reversed term
        conjugated_coefficient = coefficient.conjugate()
        conjugate_operator += QubitOperator(term, conjugated_coefficient)
    return conjugate_operator

# Function to enforce Hermiticity by symmetrizing the Hamiltonian
def make_hermitian(qubit_operator):
    # The Hermitian operator is (H + H†) / 2
    hermitian_hamiltonian = (qubit_operator + hermitian_conjugate(qubit_operator)) / 2
    return hermitian_hamiltonian


# Function to perform calculations for a given bond distance
def calculate_and_save(bond_distance, output_dir, active_electrons, active_orbitals):
    display(Markdown(f"### Processing bond distance: {bond_distance} Å"))

    # Define molecule geometry
    geometry = [('Al', (0, 0, 0)), ('H', (0, 0, bond_distance))]
    basis = 'sto-3g'
    charge = 0
    multiplicity = 1
    spin = multiplicity - 1

    # Setup molecule and perform SCF calculation
    mol = gto.M(atom=geometry, basis=basis, charge=charge, spin=spin, unit='angstrom')
    mf = scf.RHF(mol).run()

    # Perform CASCI calculation to define active space
    mc = mcscf.CASCI(mf, active_orbitals, active_electrons)
    mc.kernel()

    print(f"Bond distance: {bond_distance:.2f} Å, HF Energy: {mf.e_tot:.8f} Hartree, CASCI Energy: {mc.e_tot:.8f} Hartree")

    # Generate molecular Hamiltonian in full space
    molecular_hamiltonian = generate_molecular_hamiltonian(
        geometry=geometry,
        basis=basis,
        multiplicity=multiplicity,
        charge=charge,
        n_active_electrons=active_electrons,
        n_active_orbitals=active_orbitals

    )
    fermion_hamiltonian = get_fermion_operator(molecular_hamiltonian)


    # Apply the Jordan-Wigner transformation
    jw_hamiltonian = jordan_wigner(fermion_hamiltonian)

    # Ensure the Jordan-Wigner Hamiltonian is Hermitian
    jw_hamiltonian = make_hermitian(jw_hamiltonian)

    # Apply the Bravyi-Kitaev transformation
    bk_hamiltonian = bravyi_kitaev_tree(fermion_hamiltonian)

    # Ensure the Bravyi-Kitaev Hamiltonian is Hermitian
    bk_hamiltonian = make_hermitian(bk_hamiltonian)

    # Save results
    save_results(bond_distance, output_dir, fermion_hamiltonian, jw_hamiltonian, bk_hamiltonian)


# Main script
output_dir = "LiH_Full"
bond_distances = [0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 1.0, 1.05, 1.10, 1.15, 1.20, 1.25, 1.30, 1.35, 1.40, 1.45, 1.50, 1.55, 1.60, 1.65, 1.70, 1.75, 1.80, 1.85, 1.90, 1.95, 2.00, 2.05, 2.10, 2.15, 2.20, 2.25, 2.30, 2.35, 2.35, 2.40, 2.45, 2.50, 2.55, 2.60, 2.65, 2.70, 2.75, 2.80, 2.85, 2.90, 2.95, 3.00]  # Specify bond distances in Å
active_electrons = 4  # Specify the number of active electrons
active_orbitals = 3   # Specify the number of active orbitals

for bond_distance in bond_distances:
    calculate_and_save(bond_distance, output_dir, active_electrons, active_orbitals)


### Processing bond distance: 0.5 Å

converged SCF energy = -237.38378808817
CASCI E = -237.426766044325  E(CI) = -3.43229327375488  S^2 = 2.0000000
Bond distance: 0.50 Å, HF Energy: -237.38378809 Hartree, CASCI Energy: -237.42676604 Hartree
Results saved in folder: LiH_Full/AlH_50


### Processing bond distance: 0.55 Å

converged SCF energy = -237.763058702601
CASCI E = -237.791231021523  E(CI) = -3.45213082672493  S^2 = 2.0000000
Bond distance: 0.55 Å, HF Energy: -237.76305870 Hartree, CASCI Energy: -237.79123102 Hartree
Results saved in folder: LiH_Full/AlH_55


### Processing bond distance: 0.6 Å

...

converged SCF energy = -239.300480264991
CASCI E = -239.314546665447  E(CI) = -2.61317962502139  S^2 = 0.0000000
Bond distance: 2.90 Å, HF Energy: -239.30048026 Hartree, CASCI Energy: -239.31454667 Hartree
Results saved in folder: LiH_Full/AlH_290


### Processing bond distance: 2.95 Å

converged SCF energy = -239.293899810491
CASCI E = -239.307969777645  E(CI) = -2.59731817647818  S^2 = 0.0000000
Bond distance: 2.95 Å, HF Energy: -239.29389981 Hartree, CASCI Energy: -239.30796978 Hartree
Results saved in folder: LiH_Full/AlH_295


### Processing bond distance: 3.0 Å

converged SCF energy = -239.287571549211
CASCI E = -239.301644920313  E(CI) = -2.58201742682351  S^2 = 0.0000000
Bond distance: 3.00 Å, HF Energy: -239.28757155 Hartree, CASCI Energy: -239.30164492 Hartree
Results saved in folder: LiH_Full/AlH_300


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_algorithms.minimum_eigensolvers import AdaptVQE, VQE


# Display step function
def display_step(message):
    print(message)

display_step("Step 1: Define the LiH molecule geometry and basis set")
basis = 'sto-3g'  # Minimal basis set
multiplicity = 1  # Singlet state (closed shell)
charge = 0  # Neutral molecule

# Define distances for the geometry
distances = [x * 0.05 for x in range(6, 60)]  # Distances from 0.3 Å to 3.0 Å

energies = []
hf_energies = []

# Output files
output_log_file = "results_log.txt"
output_data_file = "bond_distance_vs_energy.txt"
output_plot_file = "energy_profile.png"

# Estimator instance for VQE
estimator = Estimator()

# Open the log file
with open(output_log_file, "w") as log_file:
    log_file.write("Bond Distance (Å)    VQE Energy (Hartree)    HF Energy (Hartree)\n")

print(f'Processing step __', end='')
for i, d in enumerate(distances):
    print('\b\b{:2d}'.format(i), end='', flush=True)
    try:
        # Update geometry with current distance
        geometry = [('Li', (0, 0, 0)), ('H', (0, 0, d))]
        driver = PySCFDriver(
            atom='; '.join([f'{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}' for atom in geometry]),
            basis=basis,
            charge=charge,
            spin=multiplicity - 1
        )
        molecule = driver.run()

        # Freeze core orbitals if applicable
        transformer = FreezeCoreTransformer()
        molecule = transformer.transform(molecule)

        # Generate the Hamiltonian
        hamiltonian = molecule.hamiltonian.second_q_op()

        # Use ParityMapper
        mapper = ParityMapper()

        # Set up the ansatz and VQE
        optimizer = SLSQP(maxiter=10000, ftol=1e-9)
        ansatz = UCCSD(
            molecule.num_spatial_orbitals,
            molecule.num_particles,
            mapper,
            initial_state=HartreeFock(
                molecule.num_spatial_orbitals,
                molecule.num_particles,
                mapper,
            ),
        )
        vqe = VQE(estimator=estimator, ansatz=ansatz, optimizer=optimizer)
        algo = GroundStateEigensolver(mapper, vqe)
        result = algo.solve(molecule)

        # Store energies
        energies.append(result.total_energies[0])
        hf_energies.append(result.hartree_fock_energy)

        # Write results to the log file
        with open(output_log_file, "a") as log_file:
            log_file.write(f"{d:.2f}                 {result.total_energies[0]:.8f}             {result.hartree_fock_energy:.8f}\n")

    except Exception as e:
        print(f"Error at distance {d:.2f} Å: {e}")
        with open(output_log_file, "a") as log_file:
            log_file.write(f"{d:.2f} Å: Error occurred - {e}\n")

print(' --- complete')

# Check if energies list is empty
if not energies or not hf_energies:
    print("Error: No energies were calculated. Please check the loop and debug.")
    exit()

# Find the lowest VQE and Hartree-Fock energies
lowest_vqe_energy = min(energies)
lowest_hf_energy = min(hf_energies)

# Find the corresponding distances
lowest_vqe_distance = distances[energies.index(lowest_vqe_energy)]
lowest_hf_distance = distances[hf_energies.index(lowest_hf_energy)]

# Save results to a data file
with open(output_data_file, "w") as data_file:
    data_file.write("Bond Distance (Å)\tVQE Energy (Hartree)\tHF Energy (Hartree)\n")
    for d, vqe, hf in zip(distances, energies, hf_energies):
        data_file.write(f"{d:.2f}\t{vqe:.8f}\t{hf:.8f}\n")
    data_file.write(f"\nLowest VQE Energy: {lowest_vqe_energy:.8f} Hartree at bond distance {lowest_vqe_distance:.2f} Å\n")
    data_file.write(f"Lowest Hartree-Fock Energy: {lowest_hf_energy:.8f} Hartree at bond distance {lowest_hf_distance:.2f} Å\n")

print(f"Results saved to {output_data_file}")

# Print the lowest energies and corresponding bond distances
print(f"Lowest VQE Energy: {lowest_vqe_energy:.8f} Hartree at bond distance {lowest_vqe_distance:.2f} Å")
print(f"Lowest Hartree-Fock Energy: {lowest_hf_energy:.8f} Hartree at bond distance {lowest_hf_distance:.2f} Å")

# Plotting the energy profile
fig, ax = plt.subplots()
ax.plot(distances, energies, label="VQE Energy")
ax.plot(distances, hf_energies, label="Hartree-Fock Energy", linestyle='--')
ax.set_xlabel("Bond Distance (Å)")
ax.set_ylabel("Energy (Hartree)")
ax.legend()
plt.savefig(output_plot_file)  # Save the plot
plt.show()

print(f"Energy profile plot saved as {output_plot_file}")


Step 1: Define the LiH molecule geometry and basis set
Processing step  0

/tmp/ipykernel_1885599/3315098766.py:34: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


 1

KeyboardInterrupt: 